In [1]:
## Prediction 

## specify train data
train_data = ""
## specify validation data
dataset = ""
text = ""

## model_no (cf. training.py)
model_no = 0
# model from which epoch?
epoch = 1

In [2]:
### importing libraries
import torch  # ML lib. allows computation of tensors on GPUs
import pandas as pd  # data analysis and manipulation
import numpy as np  # for mathematical operations
from transformers import BertTokenizer  # tokenizes sequences of strings
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import SequentialSampler
from transformers import BertForSequenceClassification  # our model

OSError: [WinError 127] Die angegebene Prozedur wurde nicht gefunden. Error loading "C:\Users\NALMPI\anaconda3\lib\site-packages\torch\lib\c10_cuda.dll" or one of its dependencies.

In [3]:
# Load data 

# read data
df = pd.read_csv("./data/processed/{}.csv".format(dataset))

# load model specs
with open(
    "./data/temp/training/{}/model_parameters_{}.json".format(dataset, model_no)
) as f:
    model_parameters = json.load(f)

NameError: name 'pd' is not defined

In [4]:
model_type = model_specs["model_type"]
MAX_LEN = model_specs["MAX_LEN"]
BATCH_SIZE = model_specs["BATCH_SIZE"]
LR = model_specs["LR"]
EPS = model_specs["EPS"]
device_name = model_specs["device_name"]



NameError: name 'model_specs' is not defined

In [5]:
# specify device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# define tokenizers
tokenizer = BertTokenizer.from_pretrained(model_type, do_lower_case=True)

NameError: name 'torch' is not defined

In [6]:
## encode text data
encoded_data_val = tokenizer.batch_encode_plus(
    df[text].values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=MAX_LEN,
    truncation=True,
    return_tensors="pt",
)

## prepare input for model
input_ids_val = encoded_data_val["input_ids"]
attention_masks_val = encoded_data_val["attention_mask"]

# TensorDataset combines prepared model inputs in a tensor for the model
dataset_val = TensorDataset(input_ids_val, attention_masks_val)

## load the number of labels required for initial model
NUM_LABELS = np.load(
    "./data/temp/training/{}/num_labels.npy".format(train_data)
)

NameError: name 'tokenizer' is not defined

In [7]:
model = BertForSequenceClassification.from_pretrained(
    model_type,
    num_labels=NUM_LABELS,
    output_attentions=False,
    output_hidden_states=True,
)

## overwrite model parameters with finetuned parameters
model.load_state_dict(
    torch.load(
        f"./results/sentiment_analysis/training/{train_data}/BERT_{model_no}_epoch_{epoch}.model"
    )
)

model.to(device)
model.eval()
with torch.no_grad():
    outputs = model(input_ids_val, attention_masks_val)
logits = outputs[0]

NameError: name 'BertForSequenceClassification' is not defined

In [ ]:
predictions_flat = np.argmax(logits, axis=1).flatten()